In [1]:
import sys
import pandas as pd

sys.path.append("..")

import numpy as np
import pandas as pd
from scipy.stats import kendalltau
from xai_ranking.benchmarks import (
    human_in_the_loop,
    hierarchical_ranking_explanation,
    lime_experiment,
    shap_experiment,
    sharp_experiment,
    participation_experiment,
)
from xai_ranking.preprocessing import (
    preprocess_atp_data,
    preprocess_csrank_data,
    preprocess_higher_education_data,
)
from xai_ranking.datasets import (
    fetch_atp_data,
    fetch_csrank_data,
    fetch_higher_education_data,
    fetch_movers_data,
)
from xai_ranking.scorers import (
    atp_score,
    csrank_score,
    higher_education_score,
)
from xai_ranking.metrics import (
    max_sensitivity
)

RNG_SEED = 42

In [2]:
datasets = [
    {
        "name": "ATP",
        "data": fetch_atp_data(),
        "preprocess": preprocess_atp_data,
        "scorer": atp_score,
    },
]
xai_methods = [
    {"name": "LIME", "experiment": lime_experiment},
    {"name": "SHAP", "experiment": shap_experiment},
    {"name": "ShaRP", "experiment": sharp_experiment},
    # {"name": "Participation", "experiment": participation_experiment},
    {"name": "HRE", "experiment": hierarchical_ranking_explanation},
    {"name": "HIL", "experiment": human_in_the_loop},
]

In [35]:
def compute_faithfulness(features_df, contri_df, target_idx, score_function):
    """
    Computes faithfulness of explanation method for the point of interest
    """
    target_point = features_df.iloc[target_idx].values
    target_point_idx = features_df.iloc[target_idx].name   # actual index in df

    # Feature mean values
    means = features_df.mean().values

    # get contributions of target
    target_point_contri = contri_df.loc[target_point_idx].values

    # get outcome of target
    original_outcome = score_function(target_point) # <- This should be the qoi 

    # print("Point: ", target_point)
    # print("Means: ", means)
    # print("Contributions: ", target_point_contri)
    # print("Contribution sum: ", target_point_contri.sum())
    # print("Outcome: ", original_outcome)

    faithfulness = 0
    for ftr in range(len(means)):
        new_target_point = target_point.copy()
        new_target_point[ftr] = means[ftr]
        # print("New point: ", new_target_point)
        new_outcome = score_function(new_target_point) # <- This should be the qoi
        faithfulness += original_outcome - new_outcome - target_point_contri[ftr]

    faithfulness = faithfulness/len(means)

    # print("Result:", faithfulness)

    return faithfulness

In [36]:
features_data, _, _ = preprocess_atp_data(fetch_atp_data())

for xai_method in xai_methods:
    print("\n\n\n####################",xai_method)
    experiment_result = []
    contri_data = pd.read_csv(f"results/_contributions_ATP_{xai_method['name']}.csv", index_col="player_name")

    for target_idx in range(len(features_data)):
        experiment_result.append(compute_faithfulness(features_data, contri_data, target_idx, atp_score))

    # result_df = pd.DataFrame(experiment_result, columns=["max_sensitivity"], index=features_data.index)
    # result_df.to_csv(f"results/_max-sensitivity_ATP_{xai_method['name']}_close-any-outcome.csv")
    print(experiment_result)




#################### {'name': 'LIME', 'experiment': <function lime_experiment at 0x14ae5a3856c0>}
[array([1.40968599]), array([0.2897017]), array([0.57424456]), array([-0.35505326]), array([-1.24377554]), array([-0.33668081]), array([-1.60172971]), array([-1.42317362]), array([-0.88257143]), array([-1.12640337]), array([-1.72441542]), array([-0.9585178]), array([-1.8829994]), array([-2.08328358]), array([-1.29495351]), array([-1.39541589]), array([-1.85281992]), array([-0.69880405]), array([0.04184881]), array([-2.62291037]), array([-2.39332434]), array([-1.91244804]), array([-0.29140562]), array([0.44958114]), array([-0.38327878]), array([-0.87236024]), array([-0.79507108]), array([-1.30534832]), array([-2.46303478]), array([-1.09654901]), array([-0.56226571]), array([0.57754159]), array([0.3725393]), array([-0.09411822]), array([-1.47908116]), array([0.2481152]), array([-1.35552497]), array([-0.27445977]), array([0.53404072]), array([-1.07191955]), array([0.67383584]), array([0.02

In [57]:
def compute_fidelity(features_df, contri_df, target_idx, score_function, threshold, perturbation):
    """
    Computes faithfulness of explanation method for the point of interest
    """
    target_point = features_df.iloc[target_idx].values
    target_point_idx = features_df.iloc[target_idx].name   # actual index in df

    # Feature mean values
    stds = features_df.std().values

    # get contributions of target
    target_point_contri = contri_df.loc[target_point_idx].values

    # get outcome of target
    original_outcome = score_function(target_point)

    # print("Point: ", target_point)
    # print("Std: ", stds)
    # print("Contributions: ", target_point_contri)
    # print("Contribution sum: ", target_point_contri.sum())
    # print("Outcome: ", original_outcome)

    fidelity = 0
    contr_sum = 0
    contr_sum_mod = 100/target_point_contri.sum()
    ftr_counter=0
    for ftr in range(len(stds)):
        # Change insignificant features
        if contr_sum > threshold:
            # print(ftr," is insignificant")
            new_target_point = target_point.copy()
            new_target_point[ftr] = new_target_point[ftr] + perturbation * stds[ftr]
            # print("New point: ", new_target_point)
            new_outcome = score_function(new_target_point)
            # print("New outcome: ", new_outcome)
            fidelity += fidelity + original_outcome - new_outcome
            ftr_counter += 1
        # skip significant features
        else:
            contr_sum += target_point_contri[ftr] * contr_sum_mod
            # print(ftr," is significant")

    if ftr_counter > 0:
        fidelity = fidelity/ftr_counter
    
    # print("Result:", fidelity)

    return fidelity

In [73]:
for xai_method in xai_methods:
    print("\n\n\n####################",xai_method)
    experiment_result = []
    contri_data = pd.read_csv(f"results/_contributions_ATP_{xai_method['name']}.csv", index_col="player_name")

    for target_idx in range(len(features_data)):
        experiment_result.append(compute_fidelity(features_data, contri_data, target_idx, atp_score, 80, 0.15))

    # result_df = pd.DataFrame(experiment_result, columns=["max_sensitivity"], index=features_data.index)
    # result_df.to_csv(f"results/_max-sensitivity_ATP_{xai_method['name']}_close-any-outcome.csv")
    print(sum(abs(number) for number in experiment_result)/len(experiment_result))




#################### {'name': 'LIME', 'experiment': <function lime_experiment at 0x14ae5a3856c0>}
[0.50947769]



#################### {'name': 'SHAP', 'experiment': <function shap_experiment at 0x14ae4c8cda80>}
[0.62649895]



#################### {'name': 'ShaRP', 'experiment': <function sharp_experiment at 0x14ae4c8fed40>}
[0.51837979]



#################### {'name': 'HRE', 'experiment': <function hierarchical_ranking_explanation at 0x14ae5ac445e0>}
[0.47374163]



#################### {'name': 'HIL', 'experiment': <function human_in_the_loop at 0x14ae5abf8ae0>}
[0.3248214]


In [34]:
# max sensitivity experiment
# all points in ATP dataset, all methods

num_neighbors = 8
features_data, _, _ = preprocess_atp_data(fetch_atp_data())

for xai_method in xai_methods:
    experiment_result = []
    contri_data = pd.read_csv(f"results/_contributions_ATP_{xai_method['name']}.csv", index_col="player_name")

    for target_idx in range(len(features_data)):
        experiment_result.append(max_sensitivity(features_data, contri_data, target_idx, num_neighbors, atp_score, kendalltau))

    result_df = pd.DataFrame(experiment_result, columns=["max_sensitivity"], index=features_data.index)
    result_df.to_csv(f"results/_max-sensitivity_ATP_{xai_method['name']}_close-any-outcome.csv")

In [1]:
# contribution results
results = {}
for dataset in datasets:
    results[dataset["name"]] = {}
    for xai_method in xai_methods:
        experiment_func = xai_method["experiment"]
        preprocess_func = dataset["preprocess"]
        score_func = dataset["scorer"]
        X, ranks, scores = preprocess_func(dataset["data"])
        contributions = experiment_func(X, score_func)
        results[dataset["name"]][xai_method["name"]] = contributions
        
        result_df = pd.DataFrame(contributions, columns=X.columns, index=X.index)
        result_df.to_csv(f"results/_contributions_{dataset['name']}_{xai_method['name']}.csv")
        # with open(f"_contributions_{dataset['name']}_{xai_method['name']}.npy", "wb") as f:
        #     np.save(f, contributions)

NameError: name 'datasets' is not defined